In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, validation_curve
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import urllib.request




In [5]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/Shemeazza/ML4DS-D/main/"
WATER_BILL_PATH = os.path.join("datasets", "Water Bill")
WATER_BILL_URL = DOWNLOAD_ROOT + WATER_BILL_PATH 


def fetch_water_bill_data(water_bill_url=WATER_BILL_URL, water_bill_path=WATER_BILL_PATH):
    if not os.path.isdir(water_bill_path):
        os.makedirs(water_bill_path)
        urllib.request.urlretrieve(water_bill_url, csv_path)    

def load_training_data(water_bill_path=WATER_BILL_PATH):
    csv_path = os.path.join(water_bill_path, "train.csv")
    return pd.read_csv(csv_path)

train_data = load_training_data()


In [6]:

# train_data = pd.read_csv(train_data_path)

train_data['YearMonth'] = train_data['Year'].astype(str) + train_data['Month'].astype(str).str.zfill(2)

x = train_data[['Year', 'Month', 'Consumption', 'Installation_zone']]
y = train_data['Consumer_type']



In [8]:
%matplotlib inline

mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize = 12)
mpl.rc('ytick', labelsize = 12)
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300 ):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format = fig_extension, dpi = resolution)    


In [9]:
encoded_labels = LabelEncoder()
x.loc[:, 'Installation_zone'] = encoded_labels.fit_transform(x['Installation_zone'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)



(263980, 4)
(65995, 4)


In [10]:
model = tree.DecisionTreeClassifier(random_state=42).fit(x_train,y_train)

y_test_pred = model.predict(x_test)


In [11]:
acc_test = accuracy_score(y_test, y_test_pred)
print("\nModel Acc:", acc_test)




Model Acc: 0.9058868096067884


In [13]:
def load_competition_data(water_bill_path=WATER_BILL_PATH):
    csv_path = os.path.join(water_bill_path, "competition.csv")
    return pd.read_csv(csv_path)

In [15]:
competition_data = load_competition_data()

for col in ['Installation_zone']:
    unknown_labels = set(competition_data[col]) - set(encoded_labels.classes_)
    if unknown_labels:
        encoded_labels.classes_ = np.concatenate([encoded_labels.classes_, ['unknown_label']])
        competition_data[col] = competition_data[col].replace(unknown_labels, 'unknown_label')



In [16]:
competition_data['YearMonth'] = competition_data['Year'].astype(str) + competition_data['Month'].astype(str).str.zfill(2)
competition_data.loc[:, 'Installation_zone'] = encoded_labels.transform(competition_data['Installation_zone'])

In [17]:
x_comp = competition_data[['Year', 'Month', 'Consumption', 'Installation_zone']]

missing = set(x_train.columns) - set(x_comp.columns)
if missing:
    raise ValueError(f"Missing features: {missing}")


In [18]:
prediction = model.predict(x_comp)
results_data = pd.DataFrame({
    'Consumer_number': competition_data['Consumer_number'],
    'Consumer_type': prediction
})

In [19]:
results_data.to_csv("predicted_results.csv", index=False)
